In [1]:
import dash
from dash import dcc
from dash import html
import pandas as pd
import jupyter_dash
from jupyter_dash import JupyterDash
import dash
import folium
from folium import plugins
import pandas as pd
import geopandas as gpd
import requests
import json
import matplotlib


In [2]:
###load main data###
#Sensor data
response1=requests.get("http://localhost:5005/api/Allsensors_data")
raw_data1=response1.text
data1=json.loads(raw_data1)
data_sensors=pd.json_normalize(data1)
data_sensors_geodf = gpd.GeoDataFrame(data_sensors, geometry=gpd.points_from_xy(data_sensors['lng'], data_sensors['lat'],crs='epsg:4326'))

#AQ data
response2=requests.get("http://localhost:5005/api/AllAQ_data")
raw_data2=response2.text
data2=json.loads(raw_data2)
AQdata=pd.json_normalize(data2)
AQdata['Date'] = pd.to_datetime(AQdata['Date'], unit='ms')
AQdata = AQdata.reset_index(drop=True)

#merged data
response3=requests.get("http://localhost:5005/api/all_data")
data3=response3.json()
all_data=pd.json_normalize(data3)
all_data_geopd= gpd.GeoDataFrame(all_data, geometry=gpd.points_from_xy(all_data['lng'], all_data['lat'],crs='epsg:4326'))
all_data_geopd['Date'] = pd.to_datetime(AQdata['Date'], unit='ms')
all_data_geopd = all_data_geopd.reset_index(drop=True)

print("imported")

imported


In [3]:
######## prepare figures and plots so we can refer to them in layout#######
import matplotlib.pyplot as plt
import folium
import contextily as cx
import ipywidgets as widgets
from folium.plugins import Fullscreen
from folium.plugins import MousePosition
from dash.dependencies import Input, Output
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
import base64
import io
import plotly.io as pio


###Follium map:  m is the figure of map, so we refer to m as our folium map
location=data_sensors_geodf['lat'].mean(),data_sensors_geodf['lng'].mean()
m = folium.Map(location=location, zoom_start=8, tiles='openstreetmap')

for index, row in data_sensors_geodf.iterrows():
    folium.Marker(
        location=[row['geometry'].y,row['geometry'].x], 
        popup=f"Station{row['StationID']}:{row['StationName']},  sensor:{row['SensorType']}",
        icon=folium.map.Icon(color='red')
    ).add_to(m)
m_html = m.get_root().render()   
    
    
###heatmap :  n is the figure of the heat map 
location=data_sensors_geodf['lat'].mean(),data_sensors_geodf['lng'].mean()
n = folium.Map(location = location, tiles='Cartodb dark_matter', zoom_start = 10)
heat_data = [[point.xy[1][0], point.xy[0][0]] for point in data_sensors_geodf.geometry]
heat_data
plugins.HeatMap(heat_data).add_to(n)
n_html = n.get_root().render()



### min /max / mean

#calculating min of each sensor
min=all_data_geopd[['Average Value','SensorID']].groupby('SensorID', axis=0).min()
a={'Average Value':'min of sensor'}
min=min.rename(columns=a)
#print(min)
#min.plot(kind='bar',figsize=(30,20))

#calculating max of each sensor 
max=all_data_geopd[['SensorID','Average Value']].groupby('SensorID', axis=0).max()
b={'Average Value':'max of sensor'}
max=max.rename(columns=b)
#print(max)
#max.plot(kind='bar',figsize=(30,20))

#calculating mean of each sensor 
mean=all_data_geopd[['SensorID','Average Value']].groupby('SensorID', axis=0).mean(numeric_only=True)
c={'Average Value':'mean of sensor'}
mean=mean.rename(columns=c)
#mean.plot(kind='bar',figsize=(30,20))

#create the figures and layout for min/max/mean,because the input of dcc.Graph is a defined figure and layout
import plotly.graph_objs as go
min_plot = go.Figure(data=[
    go.Bar(x=min.index, y=min['min of sensor'])
])
min_plot.update_layout(title='Minimum Values', xaxis_title='Sensor ID', yaxis_title='Minimum Value')

# Plot for maximum values
max_plot = go.Figure(data=[
    go.Bar(x=max.index, y=max['max of sensor'])
])
max_plot.update_layout(title='Maximum Values', xaxis_title='Sensor ID', yaxis_title='Maximum Value')

# Plot for mean values
mean_plot = go.Figure(data=[
    go.Bar(x=mean.index, y=mean['mean of sensor'])
])
mean_plot.update_layout(title='Mean Values', xaxis_title='Sensor ID', yaxis_title='Mean Value')














# Grouping options for dropdown menu
province_options = data_sensors_geodf['Province'].unique()
sensor_type_options = data_sensors_geodf['SensorType'].unique()
grouping_options = ['Province: ' + option for option in province_options] + ['Sensor Type: ' + option for option in sensor_type_options] + ['All Sensors']

# Dropdown widget for the grouping options
grouping_dropdown = widgets.Dropdown(
    options=list(grouping_options),
    description='Group by:'
)
display(grouping_dropdown)


def update_map(selected_grouping_option):
    print(f"Selected Grouping Option: {selected_grouping_option}")

    # Check if the selected grouping option is valid
    if selected_grouping_option == "All Sensors":
        filtered_data = data_sensors_geodf
    else:
        if ":" not in selected_grouping_option:
            print("Please select at least one grouping option.")
            return
        grouping_type, grouping_value = selected_grouping_option.split(': ')
        if grouping_type == "Sensor Type":
            grouping_type = "SensorType" 
        filtered_data = data_sensors_geodf[data_sensors_geodf[grouping_type] == grouping_value]

    # Check availability of data
    if len(filtered_data) == 0:
        print("No data available.")
        return

    # Folium map
    l = folium.Map(location=[filtered_data['geometry'].y.mean(), filtered_data['geometry'].x.mean()], tiles='OpenStreetMap', zoom_start=7, control_scale=True) 
    
    # Fullscreen control
    fullscreen = Fullscreen()
    fullscreen.add_to(l)

    # Popup
    for index, row in filtered_data.iterrows():
        popup_content = f"<b>SensorID:</b> {row['SensorID']}<br>" \
                        f"<b>SensorType:</b> {row['SensorType']}<br>" \
                        f"<b>Sensor Province:</b> {row['Province']}<br>" \
                        f"<b>Comune:</b> {row['Comune']}"
        folium.Marker(
            location=[row['geometry'].y, row['geometry'].x],
            popup=popup_content,
            icon=folium.map.Icon(color='red')
        ).add_to(l)
    
    #l.save('map.html')
    #l_html = open('map.html').read()
    return l._repr_html_()


# Add title
title_html = '''
    <h3 align="center" style="font-size:20px"><b>Sensors Distribution</b></h3>
'''















##########second part
# Grouping options for dropdown menu
options_sensor = all_data_geopd['SensorID'].unique()


   



####__________________________Layout__________________________________####

app = jupyter_dash.JupyterDash(__name__)

app.layout = html.Div(
    style={'backgroundColor': '#EBF6FC'},
    children=[
        html.H1('Overview of Stations', style={'textAlign': 'center', 'marginTop': '0', 'backgroundColor': '#FFD98E', 'border': '2px solid black', 'padding': '10px'}),
        html.Div(
            style={'display': 'flex', 'width': '100%'},
            children=[
                html.Div(
                    style={'flex': '1','marginLeft': '100px'},
                    children=[
                        html.H2('Stations and Sensor Types'),
                        html.Iframe(srcDoc=m_html, width='80%', height='400')
                    ]
                ),
                html.Div(
                    style={'flex': '1', 'marginLeft': '20px'},
                    children=[
                        html.H2('Stations Density'),
                        html.Iframe(srcDoc=n_html, width='80%', height='400')
                    ]
                )
            ]
        ),
        html.Div(
            style={'border': '1px solid black', 'padding': '10px', 'margin': '20px', 'border': '2px solid black'},
            children=[
                html.H2('Statistics',style={'textAlign': 'center', 'marginTop': '0', 'backgroundColor': '#FFD98E', 'border': '2px solid black', 'padding': '10px'}),
                html.Div(
                    children=[
                        html.Div(
                            style={'marginBottom': '20px'},
                            children=[
                                html.H3('Minimum Values',style={'textAlign':'center'}),
                                dcc.Graph(figure=min_plot)
                            ]
                        ),
                        html.Div(
                            style={'marginBottom': '20px'},
                            children=[
                                html.H3('Maximum Values',style={'textAlign':'center'}),
                                dcc.Graph(figure=max_plot)
                            ]
                        ),
                        html.Div(
                            children=[
                                html.H3('Mean Values',style={'textAlign':'center'}),
                                dcc.Graph(figure=mean_plot)
                            ]
                        )
                    ]
                )
            ]
        ),
        html.Div(
            style={'border': '2px solid black', 'padding': '10px', 'margin': '20px'},
            children=[
                html.H2('Sensors Map', style={'textAlign': 'center', 'marginTop': '0', 'backgroundColor': '#FFD98E', 'border': '2px solid black', 'padding': '10px'}),
                html.Div(
                    style={'marginBottom': '20px'},
                    children=[
                        html.H3('Group by:'),
                        dcc.Dropdown(
                            id='grouping-dropdown',
                            options=[
                                {'label': 'Province: ' + option, 'value': 'Province: ' + option} for option in province_options
                            ] + [
                                {'label': 'Sensor Type: ' + option, 'value': 'Sensor Type: ' + option} for option in sensor_type_options
                            ] + [
                                {'label': 'All Sensors', 'value': 'All Sensors'}
                            ],
                            value='All Sensors',
                            clearable=False
                        ),
                    ]
                ),
                html.Div(
                    id='map-container',
                    style={'height': '500px'}
                )
            ]
        ),#######
        html.Div(
    children=[
        html.H1('Pollution Value Plot', style={'textAlign': 'center', 'backgroundColor': '#FFD98E', 'border': '2px solid black', 'padding': '10px'}),
        html.Div(
            style={'display': 'flex', 'marginLeft': '10px'},
            children=[
                html.Div(
                    style={'flex': '1'},
                    children=[
                        html.Label('Sensor ID'),
                        dcc.Dropdown(
                            id='dropdown-sensor',
                            options=[{'label': str(option), 'value': option} for option in options_sensor],
                        ),
                    ]
                ),
                html.Div(
                    style={'flex': '1', 'marginLeft': '30px', 'marginTop': '10px'},
                    children=[
                        html.Label('Start Date'),
                        dcc.DatePickerSingle(
                            id='date-picker-start',
                            min_date_allowed=pd.to_datetime('2022-01-01'),
                            max_date_allowed=pd.to_datetime('2022-12-31'),
                            initial_visible_month=pd.to_datetime('2022-01-01'),
                            date=pd.to_datetime('2022-01-01')
                        ),
                    ]
                ),
                html.Div(
                    style={'flex': '1', 'marginTop': '10px'},
                    children=[
                        html.Label('End Date'),
                        dcc.DatePickerSingle(
                            id='date-picker-end',
                            min_date_allowed=pd.to_datetime('2022-01-01'),
                            max_date_allowed=pd.to_datetime('2022-12-31'),
                            initial_visible_month=pd.to_datetime('2022-12-31'),
                            date=pd.to_datetime('2022-12-31')
                        ),
                    ]
                ),
            ]
        ),
        html.Div(
            id='average-value-plot',
            style={'margin-top': '20px', 'display': 'flex', 'justify-content': 'center'}
        ),
    ]
)
    ]
    
)




    
@app.callback(
    Output('map-container', 'children'),
    Input('grouping-dropdown', 'value')
 )

def update_map_container(selected_grouping_option):
    l_html = update_map(selected_grouping_option)
    return html.Iframe(srcDoc=l_html, style={'width': '100%', 'height': '100%', 'border': 'none'})       








@app.callback(
    Output('average-value-plot', 'children'),
    Input('dropdown-sensor', 'value'),
    Input('date-picker-start', 'date'),
    Input('date-picker-end', 'date')
)
def update_average_value_plot(sensor_id, start_date, end_date):
    fig = make_subplots()

    # Convert dates to datetime
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # Filter the data for the selected sensor and date range
    filtered_data = all_data_geopd[
        (all_data_geopd['SensorID'] == sensor_id) &
        (all_data_geopd['Date'] >= start_date) &
        (all_data_geopd['Date'] <= end_date)
    ]

    if filtered_data.empty:
        return html.Div('No data available for the selected sensor and date range.')

    fig.add_trace(
        go.Scatter(
            x=filtered_data['Date'],
            y=filtered_data['Average Value'],
            mode='markers',
            marker=dict(color='red')
        )
    )

    fig.update_layout(
        xaxis=dict(
            title='Date',
            title_font=dict(size=12),
            tickformat='%b',
            tickfont=dict(size=12),
            tickangle=0
        ),
        yaxis=dict(
            title='Value',
            title_font=dict(size=12)
        ),
        title=f"Pollution Value for Sensor {sensor_id} ({start_date} to {end_date})",
        title_font=dict(size=12),
        showlegend=False,
        plot_bgcolor='#FFF380'
    )

    # Set the X-axis limits
    fig.update_xaxes(range=[start_date - pd.DateOffset(days=7), end_date + pd.DateOffset(days=7)])

    # Set the background color
    fig.update_layout(plot_bgcolor='#FFF380')

    # Save the Plotly figure as a PNG image in memory
    img_bytes = fig.to_image(format='png')

    # Create a base64 encoded string from the image bytes
    img_base64 = base64.b64encode(img_bytes).decode()

    # Generate the HTML img tag with the base64 image data
    img_html = f'<img src="data:image/png;base64,{img_base64}">'

    return html.Div([html.H3('Pollution Value Plot'), html.Div(html.Img(src=f'data:image/png;base64,{img_base64}'))])





app.run_server(port=8051, debug=True)

Dropdown(description='Group by:', options=('Province: MI', 'Province: VA', 'Province: CO', 'Province: SO', 'Pr…

Dash is running on http://127.0.0.1:8051/

Dash app running on http://127.0.0.1:8051/
Selected Grouping Option: All Sensors
Selected Grouping Option: All Sensors
Selected Grouping Option: All Sensors
